# 1. 原始方法

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

template = ChatPromptTemplate([
    ("system", """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation: """),
    ("placeholder", "{history}"),
    ("user", "{input}"),
    ("ai", "")
])

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
chain = template | chat | StrOutputParser()

resp = chain.invoke({
    "history": [
        ("human", "我周五晚上要去看电影"),
    ],
    "input": "我周五要去做什么"
})

print(resp)


你提到你周五晚上要去看电影，所以可以推测你周五会去电影院享受一部电影！你已经决定看哪部电影了吗？或者你是跟朋友一起去吗？


# 通过 ConversationChain 实现

**注：**

**该方式是以前的调用方法，目前已经非废弃，只是为了保证 Langchain 的向下兼容，依然提供。**

**现在推荐通过 `RunnableWithMessageHistory` 来实现**

In [15]:
# Deprecated
from langchain.chains.conversation.base import ConversationChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="openai/gpt-4o-mini")
conversation = ConversationChain(llm=llm)
resp = conversation.invoke({"input": "我周五晚上要去看电影"})
print(resp.get("response"), "\n\n", conversation.memory, "\n\n\n")
resp = conversation.invoke({"input": "我周五晚上要做什么？"})
print(resp.get("response"), "\n\n", conversation.memory)



那听起来很不错！你打算看哪部电影呢？最近有很多新片上映，比如《沙丘：第二部》和《黑豹：瓦坎达永存》。如果你喜欢科幻或者超级英雄电影，这些都是不错的选择！或者，你更喜欢其他类型的电影，比如喜剧或爱情片？ 

 chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='我周五晚上要去看电影', additional_kwargs={}, response_metadata={}), AIMessage(content='那听起来很不错！你打算看哪部电影呢？最近有很多新片上映，比如《沙丘：第二部》和《黑豹：瓦坎达永存》。如果你喜欢科幻或者超级英雄电影，这些都是不错的选择！或者，你更喜欢其他类型的电影，比如喜剧或爱情片？', additional_kwargs={}, response_metadata={})]) 



你周五晚上计划去看电影，所以可以期待一个愉快的夜晚！看完电影后，你可能会想去吃点东西或者和朋友聊聊电影的情节和角色。你有没有想过看完电影后要去哪里？有些人喜欢去咖啡馆或餐厅放松一下。你有什么特别的计划吗？ 

 chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='我周五晚上要去看电影', additional_kwargs={}, response_metadata={}), AIMessage(content='那听起来很不错！你打算看哪部电影呢？最近有很多新片上映，比如《沙丘：第二部》和《黑豹：瓦坎达永存》。如果你喜欢科幻或者超级英雄电影，这些都是不错的选择！或者，你更喜欢其他类型的电影，比如喜剧或爱情片？', additional_kwargs={}, response_metadata={}), HumanMessage(content='我周五晚上要做什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='你周五晚上计划去看电影，所以可以期待一个愉快的夜晚！看完电影后，你可能会想去吃点东西或者和朋友聊聊电影的情节和角色。你有没有想过看完电影后要去哪里？有些人喜欢去咖啡馆或餐厅放松一下。你有什

# 通过 `RunnableWithMessageHistory` 来实现

In [32]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

store = {}


def get_session_with_history(session_id: str) -> BaseChatMessageHistory:
    return store.setdefault(session_id, InMemoryChatMessageHistory())


chat_with_message_history = RunnableWithMessageHistory(chat_model,
                                                       get_session_with_history)

config = {"configurable": {"session_id": "example1"}}

resp = chat_with_message_history.invoke(input="我周五晚上准备去看电影", config=config)
print(resp.content, "\n")
resp = chat_with_message_history.invoke(input="我之前说我周五晚上准备去干什么", config=config)
print(resp.content)

听起来很不错！你打算看什么电影呢？或者你已经有了想看的片子吗？ 

你提到过你准备去看电影。你有没有决定好要看的电影或者影院呢？


# 配合 ChatPromptTemplate 使用

In [34]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability.",),
    ("placeholder", "{chat_history}"),
    ("human", "{user_input}",)
])

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

store = {}

chat_with_message_history = RunnableWithMessageHistory(
    prompt | chat_model,
    lambda session_id: store.setdefault(session_id, InMemoryChatMessageHistory()),
    input_messages_key="user_input",
    history_messages_key="chat_history",
)
config = {"configurable": {"session_id": "example3"}}
resp = chat_with_message_history.invoke(input={"user_input": "我周五晚上准备去看电影"}, config=config)
print(resp.content, "\n")
resp = chat_with_message_history.invoke(input={"user_input": "我之前说我周五晚上准备去干什么"}, config=config)
print(resp.content)

听起来很不错！你打算看哪部电影呢？如果需要推荐或者了解电影信息，可以告诉我哦！ 

你之前提到你准备去看电影。有什么特别的电影想看吗？或者有任何电影推荐需要吗？
